In [22]:
import requests

import yaml

class GreenAnalyzerConnector(object):

    def __init__(self, url: str = "http://localhost:8000", configs_file: str = "connector-configs.yaml", aerOS_url: str = "https://ncsrd-mvp-domain.aeros-project.eu"):

        with open(configs_file) as stream:
            self.configs = yaml.safe_load(stream)
        self.aerOS_headers = { "Authorization": self.configs["authorization"] }
                
        self.url = url
        self.aerOS_url = aerOS_url
        

    def get_aerOS_domains(self):
        return requests.get(f"{self.aerOS_url}/entities?format=simplified&type=Domain", headers = self.aerOS_headers).json()
    
    def get_aerOS_infrastructure_entities(self):
        return requests.get(f"{self.aerOS_url}/entities?type=InfrastructureElement&format=simplified", headers = self.aerOS_headers).json()

    
    def get_data_from_res(self, res, lat=None, lon=None):
        if lat and lon:
            data = {
                "latitude": lat, "longitude": lon
            }
            return requests.get(f"{self.url}/api/res/{res}/predict/", json=data).json()
            
        return requests.get(f"{self.url}/api/res/{res}/predict/").json()
    
    def get_energy_mix_from_country(self, country):
        data = {"country": country.upper()}
        return requests.get(f"{self.url}/api/country_energy_mix/", json=data).json()

    def get_energy_consumption_parameters(self, model_id):
        return requests.get(f"{self.url}/api/energy_profile/{model_id}/").json()

        
        
        # return {"dynamic": 4.0, "static": 2.0}
        # return requests.get(f"{self.url}/api/consumption/{model_id}/parameters/", json=data).json()


    def set_energy_mix_predictions(self, domain_id, predictions):
        pv_only_numbers = [int(p["pred"]) for p in predictions["pv"]]
        wind_only_numbers = [int(p["pred"]) for p in predictions["wind"]]
        total_only_numbers = [int(p["pred"]) for p in predictions["total"]]
        carbon_intensity_only_numbers = [int(p["pred"]) for p in predictions["carbon_intensity"]]

        
        res = requests.post(f"{self.aerOS_url}/entities/{domain_id}/attrs/", json = { 
            f"GreenAnalyzerEnergyMix:pv":{"value": pv_only_numbers,"type": "Property"},
            f"GreenAnalyzerEnergyMix:wind":{"value": wind_only_numbers,"type": "Property"},
            f"GreenAnalyzerEnergyMix:total":{"value": total_only_numbers,"type": "Property"},
            f"GreenAnalyzerEnergyMix:carbonIntensity":{"value": carbon_intensity_only_numbers,"type": "Property"} }, headers = self.aerOS_headers)
        
        print("Energy Mix predictions are inserted")
        return res



    def set_res_predictions(self, domain_id, res_id, predictions):
        only_numbers = [int(p["prediction"]) for p in predictions]

        print("RES predictions are inserted", only_numbers, res_id)
        res =requests.post(f"{self.aerOS_url}/entities/{domain_id}/attrs/", json = { f"GreenAnalyzerRES:{res_id}":{
                                                                                        "value": only_numbers,"type": "Property"}
                                                                            }, headers = self.aerOS_headers)
        print (res)
        return res


    
    def set_energy_profile(self, node_id, parameters):
        print(node_id, parameters)

        return requests.post(f"{self.aerOS_url}/entities/{node_id}/attrs/", json = { "GreenAnalyzerEnergyProfile:dynamic":{
                                                                                                "value": parameters["dynamic"],
                                                                                                "type": "Property"},
                                                                                    "GreenAnalyzerEnergyProfile:static":{
                                                                                                "value": parameters["static"],
                                                                                                "type": "Property"}
                                                                                    }, headers = self.aerOS_headers)

        print(f"energy consumption parameters inserted at {node_id} ({parameters})")



In [23]:
green_analyzer_connector = GreenAnalyzerConnector()

In [24]:
# Update the domain with Carbon Emission and Energy Mix

for domain in green_analyzer_connector.configs['domains']:
    country = green_analyzer_connector.configs['domains'][domain]["country"]
    domains = green_analyzer_connector.get_aerOS_domains()
    domains = [d['id'] for d in domains]
    print(domain, domains)

    if domain not in domains: continue

    # Energy Mix of a Country
    predictions = green_analyzer_connector.get_energy_mix_from_country(country)
    
    out = green_analyzer_connector.set_energy_mix_predictions(domain, predictions)
    
    
    RESs = green_analyzer_connector.configs['domains'][domain].get("RES", [])
    for res in RESs:
        model_name = res['model_name']        
        lat = res['location']["lat"]
        lon = res['location']["lat"]
        predictions = green_analyzer_connector.get_data_from_res(model_name, lat, lon)
        green_analyzer_connector.set_res_predictions(domain, model_name, predictions)
        

urn:ngsi-ld:Domain:ucy:01 ['urn:ngsi-ld:Domain:NCSRD', 'urn:ngsi-ld:Domain:ucy:01', 'urn:ngsi-ld:Domain:CloudFerro']
Energy Mix predictions are inserted


In [133]:
# Update the energy consumption profiles with static and dynamic power needs

for infra_power in green_analyzer_connector.configs['energy-profiles']:
    infra_entities = green_analyzer_connector.get_aerOS_infrastructure_entities()
    entity_ids = [entity["id"] for entity in infra_entities]
    
    if infra_power['infrastructure_element_id'] in entity_ids:
        params = green_analyzer_connector.get_energy_consumption_parameters(infra_power["energy_model_id"])
        test = green_analyzer_connector.set_energy_profile(infra_power['infrastructure_element_id'], params)


urn:ngsi-ld:InfrastructureElement:ucypi {'name': 'RPI4', 'dynamic': 0.02524124, 'static': 3.2770305}


In [25]:
out

<Response [204]>

In [17]:
[int(p["pred"]) for p in predictions["carbon_intensity"]]

[325018,
 323351,
 311639,
 317595,
 358146,
 465158,
 563035,
 595228,
 574233,
 540604,
 497677,
 444966,
 398803,
 368244,
 351741,
 344528,
 347757,
 357791,
 396002,
 460742,
 448107,
 351652,
 302255,
 294613,
 290555,
 288650,
 286892,
 303072,
 348699,
 451076,
 549194,
 573198,
 558106,
 528865,
 487358,
 440315,
 399638,
 369808,
 352272,
 343815,
 344146,
 355228,
 393115,
 458608,
 453642,
 354754,
 299254,
 285508,
 290709,
 294509,
 298698,
 323506,
 363883,
 460658,
 550768,
 568611,
 559948,
 532817,
 491437,
 449393]

In [8]:
domain

'urn:ngsi-ld:Domain:ucy:01'

In [173]:
green_analyzer_connector.get_aerOS_domains()

[{'id': 'urn:ngsi-ld:Domain:NCSRD',
  'type': 'Domain',
  'description': 'NCSRD aerOS MVP Domain',
  'publicUrl': 'https://ncsrd-mvp-domain.aeros-project.eu',
  'owner': ['urn:ngsi-ld:Organization:NCSRD'],
  'isEntrypoint': False,
  'domainStatus': 'urn:ngsi-ld:DomainStatus:Functional',
  'publicKey': 'k8hAPelai4z1wBfyJO5cDUJutyuW10kOh5ezeh7p7SE='},
 {'id': 'urn:ngsi-ld:Domain:ucy:01',
  'type': 'Domain',
  'description': 'NCSRD aerOS MVP Domain',
  'publicUrl': 'https://ncsrd-mvp-domain.aeros-project.eu',
  'owner': ['urn:ngsi-ld:Organization:NCSRD'],
  'isEntrypoint': False,
  'domainStatus': 'urn:ngsi-ld:DomainStatus:Functional'},
 {'id': 'urn:ngsi-ld:Domain:CloudFerro',
  'type': 'Domain',
  'description': 'CloudFerro Domain',
  'publicUrl': 'https://cf-mvp-domain.aeros-project.eu',
  'owner': ['urn:ngsi-ld:Organization:CloudFerro'],
  'domainStatus': 'urn:ngsi-ld:DomainStatus:Functional',
  'isEntrypoint': True,
  'publicKey': 'cCsiQ7iEClPfIwqejeCXsGkLc9WHefV8yg9i9zVqoh0='}]

In [174]:
green_analyzer_connector.set_res_predictions(domain, model_name, predictions)

RES predictions are inserted [4815, 3786, 1879, 862, 192, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 429, 760, 1783, 2281, 2444, 2378, 2069, 1373, 771, 155, 0, 0, 0, 0, 0, 0, 0] inverter3


<Response [204]>

In [118]:
test

<Response [204]>

In [81]:
params

{'dynamic': 10.0, 'static': 2.0}

In [119]:
green_analyzer_connector.get_aerOS_infrastructure_entities()

[{'id': 'urn:ngsi-ld:InfrastructureElement:ncsrd-w2',
  'type': 'InfrastructureElement',
  'domain': 'urn:ngsi-ld:Domain:NCSRD',
  'hostname': 'ncsrd-w2',
  'containerTechnology': 'Kubernetes',
  'internalIpAddress': '10.220.2.119',
  'macAddress': 'fa:c2:b1:a8:1a:2e',
  'lowLevelOrchestrator': 'urn:ngsi-ld:LowLevelOrchestrator:NCSRD:01',
  'cpuCores': 2,
  'currentCpuUsage': 8,
  'ramCapacity': 7940,
  'availableRam': 5970,
  'currentRamUsage': 1970,
  'avgPowerConsumption': 2,
  'currentPowerConsumption': 2,
  'realTimeCapable': False,
  'cpuArchitecture': 'urn:ngsi-ld:CpuArchitecture:x64',
  'operatingSystem': 'urn:ngsi-ld:OperatingSystem:Linux',
  'infrastructureElementTier': 'urn:ngsi-ld:InfrastructureElementTier:Cloud',
  'infrastructureElementStatus': 'urn:ngsi-ld:InfrastructureElementStatus:Ready',
  'location': {'type': 'Point', 'coordinates': [0, 0]}},
 {'id': 'urn:ngsi-ld:InfrastructureElement:ncsrd-rt',
  'type': 'InfrastructureElement',
  'domain': 'urn:ngsi-ld:Domain:NCSR

In [66]:
infra_power['infrastructure_element_id']

'urn:ngsi-ld:InfrastructureElement:ucypi'

In [26]:
test = requests.patch(f"{green_analyzer_connector.aerOS_url}/entityOperations/upsert?options=update", data = [
                                                                                    { "id": "urn:ngsi-ld:InfrastructureElement:pi",
                                                                                       "GreenAnalyzer:EnergyProfile:dynamic":{
                                                                                                "value": params["dynamic"],
                                                                                                "type": "Property"}
                                                                                    }])

In [36]:
node_id = "urn:ngsi-ld:InfrastructureElement:pi"
test = requests.post(f"{green_analyzer_connector.aerOS_url}/entities/{node_id}/attrs/", data = {
                                                                                          "greenanalyzer:energyprofile:dynamic": {
                                                                                                "value": params["dynamic"],
                                                                                                "type": "Property"
                                                                                          },
                                                                                          "greenanalyzer:energyprofile:static": {
                                                                                                "value": params["static"],
                                                                                                "type": "Property"
                                                                                          },
                                                                                    })

In [ ]:
"/entities/{node_id}"

In [56]:
data = {
       "id": "urn:ngsi-ld:InfrastructureElement:ucypi",
       "type": "InfrastructureElement",
       "domain": {
           "type": "Relationship",
           "object": "urn:ngsi-ld:Domain:ucy:01"
       },
       "internalIpAddress": {
           "type": "Property",
           "value": "10.106.1.194"
       },
       "macAddress": {
           "type": "Property",
           "value": ""
       },
       "lowLevelOrchestrator": {
           "type": "Relationship",
           "object": "urn:ngsi-ld:LowLevelOrchestrator:NCSRD:01"
       },
       "cpuCores": {
           "type": "Property",
           "value": 4
       },
       "currentCpuUsage": {
           "type": "Property",
           "value": 29
       },
       "ramCapacity": {
           "type": "Property",
           "value": 3790
       },
       "availableRam": {
           "type": "Property",
           "value": 2796
       },
       "currentRamUsage": {
           "type": "Property",
           "value": 994
       },
       "avgPowerConsumption": {
           "type": "Property",
           "value": 5
       },
       "currentPowerConsumption": {
           "type": "Property",
           "value": 5
       },
       "realTimeCapable": {
           "type": "Property",
           "value": "false"
       },
       "cpuArchitecture": {
           "type": "Relationship",
           "object": "urn:ngsi-ld:CpuArchitecture:arm64"
       },
       "operatingSystem": {
           "type": "Relationship",
           "object": "urn:ngsi-ld:OperatingSystem:Linux"
       },
       "infrastructureElementTier": {
           "type": "Relationship",
           "object": "urn:ngsi-ld:InfrastructureElementTier:Edge"
       },
       "infrastructureElementStatus": {
           "type": "Relationship",
           "object": "urn:ngsi-ld:InfrastructureElementStatus:Ready"
       },
       "location": {
           "type": "GeoProperty",
           "value": {
               "type": "Point",
               "coordinates": [
                   35.1600,
                   33.3770
               ]
           }
       }
   }



In [57]:
test = requests.post(f"{green_analyzer_connector.aerOS_url}/entities/", json = data, headers = green_analyzer_connector.aerOS_headers)

In [59]:
test

<Response [201]>

In [66]:
infra_power['infrastructure_element_id']

'urn:ngsi-ld:InfrastructureElement:aeros-pi'

In [67]:
entity_ids

['urn:ngsi-ld:InfrastructureElement:ncsrd-w2',
 'urn:ngsi-ld:InfrastructureElement:ncsrd-rt',
 'urn:ngsi-ld:InfrastructureElement:ncsrd-w1',
 'urn:ngsi-ld:InfrastructureElement:ncsrd-m',
 'urn:ngsi-ld:InfrastructureElement:pi',
 'urn:ngsi-ld:InfrastructureElement:aeros-2-jms6qnflylil-node-0',
 'urn:ngsi-ld:InfrastructureElement:aeros-2-jms6qnflylil-node-2',
 'urn:ngsi-ld:InfrastructureElement:aeros-2-jms6qnflylil-node-1']

In [25]:
domains = green_analyzer_connector.get_aerOS_domains()


In [28]:
[domain['id'].split(":")[-1] for domain in domains]

['NCSRD', 'CloudFerro']

In [5]:
inverter = "inverter1"
requests.get(f"{url}/api/res/{inverter}/predict/").json()

[{'date': '2024-10-15T15:00:00+03:00', 'prediction': 6476.900434570312},
 {'date': '2024-10-15T16:00:00+03:00', 'prediction': 5855.343077392578},
 {'date': '2024-10-15T17:00:00+03:00', 'prediction': 2336.1638009643557},
 {'date': '2024-10-15T18:00:00+03:00', 'prediction': 189.04958206176758},
 {'date': '2024-10-15T19:00:00+03:00', 'prediction': 0.40954201221466063},
 {'date': '2024-10-15T20:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-15T21:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-15T22:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-15T23:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-16T00:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-16T01:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-16T02:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-16T03:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-16T04:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-16T05:00:00+03:00', 'prediction': 0.0},
 {'date': '2024-10-16T06:00:00+03:00', 'p

In [14]:
data = {"country": "CYPRUS"}
requests.get(f"{url}/api/country_energy_mix/", json=data).json()

{'pv': [{'index': '2024-10-16T16:00:00+03:00',
   'predictions_pv': 223.23606432496445},
  {'index': '2024-10-16T17:00:00+03:00', 'predictions_pv': 98.18845816235964},
  {'index': '2024-10-16T18:00:00+03:00', 'predictions_pv': 8.92005098505673},
  {'index': '2024-10-16T19:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-16T20:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-16T21:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-16T22:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-16T23:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-17T00:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-17T01:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-17T02:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-17T03:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-17T04:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-17T05:00:00+03:00', 'predictions_pv': 0.0},
  {'index': '2024-10-17T06:00:00+03:00'